In [102]:
import pandas as pd
import os
import io 
import datetime
import numpy as np
import geocoder
import statsmodels as sm
import statsmodels.formula.api as smf

GOOGLE_API_KEY=os.environ['GOOGLE_API_KEY']


In [103]:
# read in from repo
covid = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

# reshape into long format
# first date is 1/22/2020, index 11
first_date_index = 11
date_cols = list(covid.columns[first_date_index:])


# reduce to min vars
covid = covid[['FIPS', 'Admin2', 'Province_State'] + date_cols]

covid = covid.set_index(['FIPS', 'Admin2', 'Province_State'])
covid = pd.DataFrame(covid.stack())
covid.columns = ['cases']
covid = covid.reset_index()
covid.columns = ['FIPS','county','state','date','cases']
covid['date'] = pd.to_datetime(covid['date'])

# get state abbreviations from google api - could have hardcoded
state_abbr = {}
for st in np.unique(covid['state']):
    results = geocoder.google(st, key = GOOGLE_API_KEY)
    state_abbr[results.state] = st
    
state_abbr


{'AL': 'Alabama',
 'AK': 'Alaska',
 None: 'Virgin Islands',
 'AZ': 'Arizona',
 'AR': 'Arkansas',
 'CA': 'California',
 'CO': 'Colorado',
 'CT': 'Connecticut',
 'DE': 'Delaware',
 'NJ': 'New Jersey',
 'DC': 'District of Columbia',
 'FL': 'Florida',
 'GA': 'Georgia',
 'MI': 'Michigan',
 'HI': 'Hawaii',
 'ID': 'Idaho',
 'IL': 'Illinois',
 'IN': 'Indiana',
 'IA': 'Iowa',
 'KS': 'Kansas',
 'KY': 'Kentucky',
 'LA': 'Louisiana',
 'ME': 'Maine',
 'MD': 'Maryland',
 'MA': 'Massachusetts',
 'MN': 'Minnesota',
 'MS': 'Mississippi',
 'MO': 'Missouri',
 'MT': 'Montana',
 'NE': 'Nebraska',
 'NV': 'Nevada',
 'NH': 'New Hampshire',
 'NM': 'New Mexico',
 'NY': 'New York',
 'NC': 'North Carolina',
 'ND': 'North Dakota',
 'OH': 'Ohio',
 'OK': 'Oklahoma',
 'OR': 'Oregon',
 'PA': 'Pennsylvania',
 'RI': 'Rhode Island',
 'SC': 'South Carolina',
 'SD': 'South Dakota',
 'TN': 'Tennessee',
 'TX': 'Texas',
 'UT': 'Utah',
 'VT': 'Vermont',
 'VA': 'Virginia',
 'WA': 'Washington',
 'WV': 'West Virginia',
 'WI': 'Wi

In [104]:
# replace state name with abbr
inv_map = {v: k for k, v in state_abbr.items()}
covid['st'] = covid['state']
covid = covid.replace({'st':inv_map})


In [105]:
# pasting in Trump rally dates from wikipedia, slightly cleaned up in google sheet
rally_string = '''
date	year	city	state
June 20	2020	Tulsa	OK
June 23	2020	Phoenix	AZ
August 17	2020	Mankato	MN
August 17	2020	Oshkosh	WI
August 18	2020	Yuma	AZ
August 20	2020	Old Forge	PA
August 28	2020	Londonderry	NH
September 3	2020	Latrobe	PA
September 8	2020	Winston-Salem	NC
September 10	2020	Freeland	MI
September 12	2020	Minden	NV
September 13	2020	Henderson	NV
September 17	2020	Mosinee	WI
September 18	2020	Bemidji	MN
September 19	2020	Fayetteville	NC
September 21	2020	Vandalia	OH
September 21	2020	Swanton	OH
September 22	2020	Pittsburgh	PA
September 24	2020	Jacksonville	FL
September 25	2020	Newport News	VA
September 26	2020	Middletown	PA
September 30	2020	Duluth	MN
October 12	2020	Sanford	FL
October 13	2020	Johnstown	PA
October 14	2020	Des Moines	IA
October 15	2020	Greenville	NC
October 16	2020	Ocala	FL
October 16	2020	Macon	GA
October 17	2020	Muskegon	MI
October 17	2020	Janesville	WI
October 18	2020	Carson City	NV
October 19	2020	Prescott	AZ
October 19	2020	Tucson	AZ
October 20	2020	Erie	PA
October 21	2020	Gastonia	NC
October 23	2020	The Villages	FL
October 23	2020	Pensacola	FL
October 24	2020	Lumberton	NC
October 24	2020	Circleville	OH
October 24	2020	Waukesha	WI
October 25	2020	Manchester	NH
October 26	2020	Allentown	PA
October 26	2020	Lititz	PA
October 26	2020	Martinsburg	PA
October 27	2020	Lansing	MI
October 27	2020	West Salem	WI
October 27	2020	Omaha	NE
October 28	2020	Bullhead City	AZ
October 28	2020	Goodyear	AZ
October 29	2020	Tampa	FL
October 30	2020	Waterford Township	MI
October 30	2020	Green Bay	WI
October 30	2020	Rochester	MN
October 31	2020	Bucks County	PA
October 31	2020	Reading	PA
October 31	2020	Butler	PA
November 1	2020	Sterling Heights	MI
November 1	2020	Dubuque	IA
November 1	2020	Rome	GA
November 1	2020	Opa-locka	FL
November 2	2020	Fayetteville	NC
November 2	2020	Melbourne	FL
November 2	2020	Scranton	PA
November 2	2020	Grand Rapids	MI
'''
rallies = pd.read_table(io.StringIO(rally_string))
rallies['rally_date'] = rallies['date'] + ', ' + rallies['year'].astype(str)
rallies['rally_date'] = pd.to_datetime(rallies['rally_date'])
rallies['city_state'] = rallies.city + ', ' + rallies.state

# geocode cities for county
cities = np.unique(rallies.city + ', ' + rallies.state)
city_county = []
for city in cities:
    results = geocoder.google(city, key = GOOGLE_API_KEY)
    city_county.append((city, results.county))

# hardcoding some independent cities
with_counties = pd.DataFrame(city_county, columns=['city_state','county'])
with_counties.loc[with_counties['city_state'] == 'Carson City, NV','county'] = 'Carson City'
with_counties.loc[with_counties['city_state'] == 'Newport News, VA','county'] = 'Newport News'
with_counties['county'] = with_counties.county.str.replace(' County', '')
rallies = pd.merge(rallies, with_counties, left_on='city_state', right_on='city_state')


In [106]:
# merge rallies and covid data
covid_rallies = pd.merge(covid, rallies, how='left', left_on=['county','st','date'], right_on=['county','state','rally_date'] )

covid_rallies = covid_rallies[['FIPS','county','st','date_x','cases','city_state','rally_date']]
covid_rallies['rally'] = (~covid_rallies.city_state.isnull()).astype(int)
covid_rallies[covid_rallies['county'] == 'Maricopa']


,FIPS,county,st,date_x,cases,city_state,rally_date,rally
30456,4013.0,Maricopa,AZ,2020-01-22,0,NaN,NaT,0
30457,4013.0,Maricopa,AZ,2020-01-23,0,NaN,NaT,0
30458,4013.0,Maricopa,AZ,2020-01-24,0,NaN,NaT,0
30459,4013.0,Maricopa,AZ,2020-01-25,0,NaN,NaT,0
30460,4013.0,Maricopa,AZ,2020-01-26,1,NaN,NaT,0
...,...,...,...,...,...,...,...,...
30733,4013.0,Maricopa,AZ,2020-10-25,154182,NaN,NaT,0
30734,4013.0,Maricopa,AZ,2020-10-26,154722,NaN,NaT,0
30735,4013.0,Maricopa,AZ,2020-10-27,155458,NaN,NaT,0
30736,4013.0,Maricopa,AZ,2020-10-28,155951,"Goodyear, AZ",2020-10-28,1


In [107]:
# create var for rally 3-7 days prior
covid_rallies['rally_shift3']= covid_rallies[['county','st','rally']].groupby(['county','st']).shift(3)
covid_rallies['rally_shift4']= covid_rallies[['county','st','rally']].groupby(['county','st']).shift(4)
covid_rallies['rally_shift5']= covid_rallies[['county','st','rally']].groupby(['county','st']).shift(5)
covid_rallies['rally_shift6']= covid_rallies[['county','st','rally']].groupby(['county','st']).shift(6)
covid_rallies['rally_shift7']= covid_rallies[['county','st','rally']].groupby(['county','st']).shift(7)

covid_rallies['recent_rally'] = ((covid_rallies['rally_shift3'] == 1) | (covid_rallies['rally_shift4'] == 1) | \
    (covid_rallies['rally_shift5'] == 1) | (covid_rallies['rally_shift6'] == 1) | (covid_rallies['rally_shift7'] == 1)).astype(int) 

# create var for rally upcoming
covid_rallies['rally_shift1']= covid_rallies[['county','st','rally']].groupby(['county','st']).shift(-1)
covid_rallies['rally_shift2']= covid_rallies[['county','st','rally']].groupby(['county','st']).shift(-2)
covid_rallies['rally_shift3']= covid_rallies[['county','st','rally']].groupby(['county','st']).shift(-3)
covid_rallies['rally_shift4']= covid_rallies[['county','st','rally']].groupby(['county','st']).shift(-4)
covid_rallies['rally_shift5']= covid_rallies[['county','st','rally']].groupby(['county','st']).shift(-5)
covid_rallies['rally_shift6']= covid_rallies[['county','st','rally']].groupby(['county','st']).shift(-6)
covid_rallies['rally_shift7']= covid_rallies[['county','st','rally']].groupby(['county','st']).shift(-7)

covid_rallies['upcoming_rally'] = ((covid_rallies['rally_shift1'] == 1) | (covid_rallies['rally_shift1'] == 1) |(covid_rallies['rally_shift3'] == 1) | (covid_rallies['rally_shift4'] == 1) | \
    (covid_rallies['rally_shift5'] == 1) | (covid_rallies['rally_shift6'] == 1) | (covid_rallies['rally_shift7'] == 1)).astype(int) 

# create new cases variable and lags
covid_rallies['cases_shift1']=covid_rallies[['county','st','cases']].groupby(['county','st']).shift(1)
covid_rallies['new_cases'] = covid_rallies.cases - covid_rallies.cases_shift1
covid_rallies['new_cases_lag'] = covid_rallies[['county','st','new_cases']].groupby(['county','st']).shift(1)
covid_rallies['new_cases_lag2'] = covid_rallies[['county','st','new_cases']].groupby(['county','st']).shift(2)

# time in days
covid_rallies['t'] = ((covid_rallies['date_x']-pd.to_datetime('2020-01-21')) / np.timedelta64(1, 'D')).astype(int)
covid_rallies['t2'] = covid_rallies.t**2
covid_rallies['t3'] = covid_rallies.t**3
covid_rallies[covid_rallies['county'] == 'Maricopa']

,FIPS,county,st,date_x,cases,city_state,rally_date,rally,rally_shift3,rally_shift4,...,rally_shift1,rally_shift2,upcoming_rally,cases_shift1,new_cases,new_cases_lag,new_cases_lag2,t,t2,t3
30456,4013.0,Maricopa,AZ,2020-01-22,0,NaN,NaT,0,0.0,0.0,...,0.0,0.0,0,NaN,NaN,NaN,NaN,1,1,1
30457,4013.0,Maricopa,AZ,2020-01-23,0,NaN,NaT,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,NaN,NaN,2,4,8
30458,4013.0,Maricopa,AZ,2020-01-24,0,NaN,NaT,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,NaN,3,9,27
30459,4013.0,Maricopa,AZ,2020-01-25,0,NaN,NaT,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,4,16,64
30460,4013.0,Maricopa,AZ,2020-01-26,1,NaN,NaT,0,0.0,0.0,...,0.0,0.0,0,0.0,1.0,0.0,0.0,5,25,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30733,4013.0,Maricopa,AZ,2020-10-25,154182,NaN,NaT,0,1.0,0.0,...,0.0,0.0,1,153177.0,1005.0,482.0,594.0,278,77284,21484952
30734,4013.0,Maricopa,AZ,2020-10-26,154722,NaN,NaT,0,0.0,NaN,...,0.0,1.0,0,154182.0,540.0,1005.0,482.0,279,77841,21717639
30735,4013.0,Maricopa,AZ,2020-10-27,155458,NaN,NaT,0,NaN,NaN,...,1.0,0.0,1,154722.0,736.0,540.0,1005.0,280,78400,21952000
30736,4013.0,Maricopa,AZ,2020-10-28,155951,"Goodyear, AZ",2020-10-28,1,NaN,NaN,...,0.0,NaN,0,155458.0,493.0,736.0,540.0,281,78961,22188041


In [108]:
covid_clean = covid_rallies[~covid_rallies['new_cases'].isna()]


results = smf.ols('new_cases ~ t + t2 + t3 + new_cases_lag + cases + cases^2 + recent_rally + rally + upcoming_rally + st ', data=covid_clean).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              new_cases   R-squared:                       0.527
Model:                            OLS   Adj. R-squared:                  0.527
Method:                 Least Squares   F-statistic:                 1.731e+04
Date:                Sat, 31 Oct 2020   Prob (F-statistic):               0.00
Time:                        00:50:30   Log-Likelihood:            -4.8458e+06
No. Observations:              933520   AIC:                         9.692e+06
Df Residuals:                  933459   BIC:                         9.692e+06
Df Model:                          60                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -2.1526      0.50

In [112]:

covid_rallies.to_csv('/data/home/futch/covid_rallies/data/covid_rallies.csv', index = False)


'/data/home/futch/scratch'